In [45]:
# https://mathoverflow.net/questions/369751/boolean-ring-of-unitary-divisors-structure-of-unitary-divisors
from sage.all import * 

def unitary_divisors(n):
    return [x for x in divisors(n) if gcd(x,n/x)==1]

def kk(a,b):
    return gcd(a,b)**2/(a*b)

def plus(a,b):
    return a*b/(gcd(a,b)**2)

def mult(a,b):
    return gcd(a,b)

def compositionToDivisor(c):
    s = Composition(c).to_subset()
    if len(s)==0:
        return 1
    return prod([nth_prime(x) for x in s])

def rad(n):
    return prod(prime_divisors(n))

def divisorToComposition(d,n):
    if rad(d)==d:
        s = set([prime_pi(x) for x in prime_divisors(d)])
        return Composition(from_subset=(s,n))

def compositionKernel(c1,c2):
    a = compositionToDivisor(c1)
    b = compositionToDivisor(c2)
    return kk(a,b)

def primorial(n):
    return prod(primes(nth_prime(n)))


def pivoted_cholesky(M,max_rank=int(30)):
    import tensorflow_probability as tfp
    import tensorflow as tf
    import numpy as np
    piv_chol = (tfp.math.pivoted_cholesky(M,max_rank = max_rank, diag_rtol=float(0.001), name=None)).numpy()
    return piv_chol


# https://mathoverflow.net/questions/373475/a-geometric-approach-to-the-odd-perfect-number-problem
# embedding:
def jordanTotient(n,k=2):
    return n**2*prod([1-1/p**2 for p in prime_divisors(n)])

# embedding:
# https://mathoverflow.net/questions/373475/a-geometric-approach-to-the-odd-perfect-number-problem
def phi(C,axis=0):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np
    n = int(np.sum(list(C)))
    print("n = ",n)
    A = compositionToDivisor(C)
    print("A =",A)
    print("P_n = ",primorial(n-1))
    if axis == 1:
        S = csr_matrix((int(1),int(2**(n-1))), dtype=np.float64)
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))            
            S[0,x]         = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
    elif axis == 0:
        S = dok_matrix((int(2**(n-1)),int(1)), dtype=np.float64)
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))            
            S[x,0]         = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
    return S    

# embedding of all Compositions(n) in a dok_matrix
def phiN(n,maxN=8):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np
    if n > maxN:
        return None
    CC = Compositions(n)
    lC = 2**(n-1)
    S = dok_matrix((lC,lC),dtype=np.float64)
    #print(S.shape)
    c = 0
    #print(CC)
    for C in CC:
        #print(C)
        A = compositionToDivisor(list(C))
        for d in divisors(A):
            ds = divisorToComposition(d,n).to_subset()
            x = int(np.sum([2**(i-1) for i in ds]))
            S[c,x] = float(1.0/A)*np.sqrt(jordanTotient(d,k=2))
        c+=1    
    return S

def embN(n,maxN=8):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np    
    if n > maxN:
        return None
    CC = Compositions(n)
    lC = 2**(n-1)
    S = dok_matrix((n,lC),dtype=np.float64)
    c =0
    for C in CC:
        sC = C.to_subset()
        for s in sC:
            S[s-1,c] = 1
        c+=1    
    return S.transpose()    

def emb(C,axis=1):
    from scipy.sparse import dok_matrix,lil_matrix,csr_matrix,csc_matrix
    import numpy as np    
    n = sum(C)
    lC = 2**(n-1)
    print(n,lC)
    if axis == 0:
        S = dok_matrix((lC,1),dtype=np.float64)
        sC = Composition(C).to_subset()
        for s in sC:
            S[s-1,0] = 1
    if axis == 1:
        S = dok_matrix((1,lC),dtype=np.float64)
        sC = Composition(C).to_subset()
        for s in sC:
            S[0,s-1] = 1
    return S 


def fit_neighbors(embN,n):
    X = embN(n,maxN=9)
    CC = list(Compositions(n))
    from sklearn.neighbors import NearestNeighbors as NN
    neigh = NN()
    neigh = neigh.fit(X)
    return neigh,CC


def get_neighbors(emb,neigh,CC,C,n_neighbors=5,reverse=True,verbose=False):
    v = emb(C,axis=0).transpose()
    dists, inds = (neigh.kneighbors(v, n_neighbors, return_distance=True))
    if verbose:
        print("dists = ",dists)
        print("inds = ", inds)
    ll = []
    for i in inds[0]:
        ll.append(CC[i])
    if reverse:
        return ll[-1::-1]
    else:    
        return ll    

neigh,CC = fit_neighbors(phiN,8)
X = phiN(4,maxN=8)
print(matrix(X).rank())
ll = get_neighbors(phi,neigh,CC,[2,1,1,2,1,1],n_neighbors=15,verbose=True,reverse=True)
print(ll)

8
n =  8
A = 23205
P_n =  30030
dists =  [[0.         1.         1.15470054 1.26491106 1.29099445 1.30930734
  1.34164079 1.34839972 1.35873244 1.36277029 1.3662601  1.37198868
  1.38013112 1.38169856 1.38675049]]
inds =  [[ 68   4 100  84  36  76  20  64  70  12 116  69 108   0   6]]
[[1, 1, 1, 1, 3, 1], [1, 1, 1, 1, 1, 1, 1, 1], [3, 3, 1, 1], [2, 1, 1, 2, 2], [4, 2, 1, 1], [1, 1, 1, 3, 1, 1], [2, 1, 1, 3, 1], [2, 1, 1, 1, 1, 1, 1], [1, 1, 2, 2, 1, 1], [2, 1, 3, 1, 1], [1, 2, 1, 2, 1, 1], [2, 2, 2, 1, 1], [3, 1, 2, 1, 1], [1, 1, 1, 1, 2, 1, 1], [2, 1, 1, 2, 1, 1]]


In [52]:

def kernPitch(k1,k2):
    q = getRational(k2-k1)
    a,b = q.numerator(),q.denominator()
    return gcd(a,b)*2/(a+b)

def kernDuration(k1,k2):
    return  min(k1,k2)/max(k1,k2)

def kernVolume(k1,k2):
    return min(k1,k2)/max(k1,k2)

def getRational(k):
    alpha = 2**(1/12.0)
    x = RDF(alpha**k).n(50)
    return x.nearby_rational(max_error=0.01*x)

def kernChord(c1,c2):
    k = 1/(len(c1)*len(c2))*sum([kernPitch(x,y) for x in c1 for y in c2])*1.0
    #print(k)
    return k


def getNearestChords(t1,chords,n_neighbors=1,reverse=True):
    s = [x[1] for x in sorted([ (kernChord(t1,t2),t2) for t2 in chords],reverse=True)]
    #print(s)
    if reverse:
        return s[:n_neighbors]
    else:
        return s[(n_neighbors-1)::-1]
    
def getNeighborsGraph(pitchlist,nn):
    import networkx as nx
    G = nx.DiGraph()
    for a in pitchlist:
        lv = getNearestChords([a],[[p] for p in pitchlist],n_neighbors=nn,reverse=True)
        for B in lv:
            b = B[0]
        #print(convertPitchNrToNote(a),(a-b)%12,convertPitchNrToNote(b))
            G.add_node(a)
            G.add_node(b)
            G.add_edge(a,b)
    return G     

pitchlist = list(range(36,88))

G7 = getNeighborsGraph(pitchlist,7)

print(G7.out_degree())

print(list(G7[60]))

getNearestChords([60],[[p] for p in pitchlist],n_neighbors=7,reverse=False)

[(36, 7), (48, 7), (55, 7), (60, 7), (43, 7), (64, 7), (67, 7), (37, 7), (49, 7), (56, 7), (61, 7), (44, 7), (65, 7), (68, 7), (38, 7), (50, 7), (57, 7), (62, 7), (45, 7), (66, 7), (69, 7), (39, 7), (51, 7), (58, 7), (63, 7), (46, 7), (70, 7), (40, 7), (52, 7), (59, 7), (47, 7), (71, 7), (41, 7), (53, 7), (72, 7), (42, 7), (54, 7), (73, 7), (74, 7), (75, 7), (76, 7), (77, 7), (78, 7), (79, 7), (80, 7), (81, 7), (82, 7), (83, 7), (84, 7), (85, 7), (86, 7), (87, 7)]
[60, 72, 48, 79, 41, 84, 67]


[[67], [84], [41], [79], [48], [72], [60]]

In [60]:
import music21 as m21
score = m21.stream.Score()
tempo  = 150
tm = m21.tempo.MetronomeMark(number=tempo)
score.append(tm)

def getPadicDigits(q=2/3,p=2,prec=100):
    p = p
    RDigits = Qp(p=p,prec=prec,type="lattice-cap",print_mode="bars",print_sep=",")
    R = RDigits
    digits = ([ int(x) for x in str(R(q)).replace(".","").split(",") if len(x)>0])
    #digits.reverse()
    return digits[0:prec]


vollist = [128//8*k for k in [2,3,4,5,6,7]]

def harmonic(t,y0,y1,periodendauer):
    T = periodendauer
    #X = matrix([[np.sin(np.pi/2+phi0),1],[np.sin(np.pi/2),1]])
    #print(X)
    A = (y0-y1)/2
    B = (y0+y1)/2
    #print(A,B)
    frequenz = 1.0/T
    return int(np.round(A*np.cos(2*np.pi*frequenz*t)+B,0))

def getHarmonicFromList(t,ll,T):
    periodendauer = T
    y0 = 0
    y1 = len(ll)-1
    return ll[harmonic(t,y0,y1,periodendauer)]

def convertPitchNrToNote(pitchNr):
    Z12ToPitch = dict(zip(range(12),["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]))
    octave = pitchNr // 12
    pitch = pitchNr % 12
    
    note = Z12ToPitch[pitch]    
    
    return note +  str(octave)

parts = []
scale = [60 + x for x in [0,2,4,5,7,9,11]]

drum = list(range(41,48+1))

class WarmPad(m21.instrument.Instrument):
    '''
    >>> p = instrument.Piano()
    >>> p.instrumentName
    'Piano'
    >>> p.midiProgram
    0
    '''

    def __init__(self):
        super().__init__()

        self.instrumentName = 'Warm Pad'
        self.instrumentAbbreviation = 'W.P.'
        self.midiProgram = 89

        self.lowestNote = m21.pitch.Pitch('A0')
        self.highestNote = m21.pitch.Pitch('C8')

        
def minkowskiSum(A,B):
    return ([a+b for a in A for b in B])

def k_Minkowski(k,A):
    if k < 0:
        return None
    
    if k == 1:
        return A
    else:
        return minkowskiSum(k_Minkowski(k-1,A),A)


def cyclicExtension(nn):
    k = len(nn)
    ll = [n for n in nn]
    for i in range(k):
        ll.extend([nn[(j+i)%k] for j in range(k)])
    return ll    

def applyPermutationToList(perm,ll):
    if len(perm)==len(ll) and sorted(list(set(perm)))==list(range(len(ll))):
        return [ll[perm[i]] for i in range(len(perm))]

def getBarNumber(duration):
    import numpy as np
    return int(np.floor(duration/4.0))

def getSmallestPeriodicRational(periodlength=1,prime=7):
    n = 2
    while True:
        if n%100==0:
            print("n = ",n,"periodlength = ",periodlength, "prime = ",prime)
        Zn = Zmod(n)
        for x in range(1,n):
            if gcd(x,prime)==1 and gcd(n,prime)==1 and multiplicative_order(Zn(prime))==periodlength:
                return(-x/n)
        n+=1    

    
def cumSum(ll):
    return [sum(ll[0:i]) for i in range(len(ll)+1)]
        
piano = m21.instrument.Piano()
woodblock = m21.instrument.Woodblock()
taikodrum = m21.instrument.Taiko()
percussion = taikodrum
warmpad = WarmPad()
marimba = m21.instrument.Marimba()
harp = m21.instrument.Harp()
string = m21.instrument.StringInstrument()
guitar = m21.instrument.AcousticGuitar()
acousticbass = m21.instrument.AcousticBass()

nrInst = 4
for k in range(nrInst):
    lh = m21.stream.Part()
    #lh.append(m21.instrument.Piano())
    #lh.append(WarmPad())
    if k%10 in [0]:
        lh.append(piano)
    elif k%10 in [1]:
        inst = piano #m21.instrument.AcousticBass()
        #inst.midiChannel = 9
        #inst.midiProgram = 53
        #inst.instrumentName = 'Orchestra Kit'
        #inst.instrumentAbbreviation = 'O.K.'
        print(inst)
   # 1 Standard Kit
   # 9 Room Kit
   # 17 Power Kit
   # 25 Electronic Kit
   # 26 TR-808 Kit
   # 33 Jazz Kit
   # 41 Brush Kit
   # 49 Orchestra Kit
   # 57 Sound FX Kit
   # 128 Percussion      
        lh.append(piano)
    elif k%10 in [2,3]:
        lh.append(piano) #m21.instrument.Piano())
    elif k%10 in [4,5]:
        lh.append(harp) #m21.instrument.Harp())
    elif k%10 in [6,7]:
        lh.append(warmpad) #m21.instrument.Marimba())
    elif k%10 in [8,9]:
        lh.append(string) #WarmPad())
    parts.append(lh)


#shuffle(parts)    
    
prec = 256*20

import numpy as np

Npartition = 5
#op = OrderedPartitions(Npartition).list()[1:]

#shuffle(op)
changeAfterNSeconds = 5
last_tim = -changeAfterNSeconds
cnt = 0
nBars = 2**6
print(nBars)
if prec < nBars:
    print("......")
    
N = 2*3**2*5
possibleParts = [2,4,8]
#print(numbers)
cnt = 0
numbers = [getSmallestPeriodicRational(periodlength=d,prime=7) for d in [6,3,2,1]]
#restsNumbers = [getSmallestPeriodicRational(periodlength=d,prime=2) for d in divisors(N) if d > 1]
#print("restsNumbers = ",restsNumbers)


lB = floor(log(nBars)/log(2))+1

print(nrInst)
onOffInsts = [range(x*nrInst//lB,(x+1)*nrInst//lB) for x in range(lB)]
instsToPlay = []
for b in range(2**lB):
    digits2 = Integer(b).digits(2,padto=lB)
    print(digits2)
    playInst = []
    for x in range(nrInst):
        if any([x in onOffInsts[i] for i in range(lB) if digits2[i]==1]):
            pI = 1
        else:
            pI = 0
        playInst.append(pI)    
    print(playInst) 
    instsToPlay.append(playInst)


cnt = 0
neigh,CC = fit_neighbors(phiN,8)
neigh9,CC9= fit_neighbors(phiN,9)
t = 0
tempolist = [60,70,90,120]

for i in range(len(parts)):
    #digits = getPadicDigits(q=numbers[i%len(numbers)],p=7,prec=prec)
    #rests = getPadicDigits(q=restsNumbers[i%len(restsNumbers)],p=2,prec=prec)
    durs = getPadicDigits(q=numbers[1],p=2,prec=prec)
    nPartitions = getPadicDigits(q=numbers[i%len(numbers)],p=3,prec=prec)
    #m = len(digits)
    #print(i,numbers[i])
    #print(digits)
    print(i,nPartitions)
    #print(durs)
    sumDur = 0
    sumNN = 0
    #cnt = 0
    if i==0:
        x = [1,1,1,1,1,1,1,1]
    elif i ==1:
        x = [2,1,1,2,1,1]
    elif i==2:
        x = [2,2,2,2]
    elif i==3:
        x = [4,2,2]
    LL = []
    cx = 0
    startPitch = 60
    rev = False
    for bn in range(1,2**lB+1):
        time_in_minutes = bn*1/tempo*1.0
        tim = int(np.round(time_in_minutes*60/changeAfterNSeconds,0)*changeAfterNSeconds)
        
        playInst = instsToPlay[bn-1]

        if bn in cumSum([1,2,4,8,4,8,4,8,16,8,16,8,4,8,4,4,4,2,8,4,2,1]):
            tempo = getHarmonicFromList(bn,tempolist,T=2*len(tempolist))
            parts[i].append(m21.tempo.MetronomeMark(number=tempo))
            digits = getPadicDigits(q=numbers[(i+cx)%len(numbers)],p=7,prec=prec)
            cx += 1
            rev = not rev
        #if len(LL)==0:
        #    ll = get_neighbors(phi,neigh,CC,x,n_neighbors=tempo,reverse=rev)
        #    LL.extend(ll)
            #op = OrderedPartitions(nPart).list()[:0:-1]
            #shuffle(op)
        #xdurs = LL.pop(0)
        if i%2 in [0,1]:
            xdurs = get_neighbors(phi,neigh,CC,x,n_neighbors=tempo,reverse=rev)[-1]
        else:    
            xdurs = get_neighbors(phi,neigh9,CC9,x,n_neighbors=tempo,reverse=rev)[-1]
        volume = getHarmonicFromList(bn,vollist,T=2*len(vollist))
        print("volume",volume,tim,time_in_minutes,bn,cnt)
            #shuffle(xdurs)
        print(xdurs,x,LL)
        #x = xdurs
        cnt += 1
        nPart = np.sum(list(xdurs))
        #last_tim = tim
        j = 0
        dd = [y[0] for y in getNearestChords([startPitch],[[p] for p in pitchlist],n_neighbors=len(xdurs),reverse=False)]
        for pitchNr in dd: #digits[sumNN:(sumNN+len(xdurs))]:
            #isRest = (rests[j]==1)
            #pitchNr = scale[digit]
            D = float(xdurs[j%len(xdurs)]*4.0/nPart)   
            j+=1
            sumDur += D#print(bn,bnDigits)                 
            
            if playInst[i]==-1:
                n0 = m21.note.Rest()
            else:    
                if i%10 in [0,1,2,3]:# violine
                    n0 = m21.note.Note(convertPitchNrToNote(pitchNr-(i%2)*24))
                else:
                    n0 = m21.note.Note(convertPitchNrToNote(pitchNr-(i%2)*24+12))
                #n0 = m21.note.Note(convertPitchNrToNote(pitchNr-24))    
                n0.volume.velocity = volume        
            duration = m21.duration.Duration(D)
            n0.duration = duration #getDurationInQuarterByEmphasis(emph,factor=factor)
            parts[i].append(n0)
        startPitch = dd[-1]    
        sumNN+=len(xdurs)

for part in parts:
    score.append(part)
            
score.insert(0,m21.metadata.Metadata())    
title = "Tempi for two pianos"
score.metadata.title = title

score.metadata.composer = 'musescore1983'
score.write("midi", fp=("_".join(title.split(" ")))+".mid")     
score.write("musicxml", fp=("_".join(title.split(" ")))+".musicxml")     
    

Piano
64
4
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0]
[1, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0]
[1, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 0]
[1, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 0]
[0, 0, 1, 1, 0, 0, 0]
[0, 1, 0, 0]
[1, 0, 1, 1, 0, 0, 0]
[0, 1, 0, 0]
[0, 1, 1, 1, 0, 0, 0]
[1, 1, 0, 0]
[1, 1, 1, 1, 0, 0, 0]
[1, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 0]
[1, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0]
[1, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0]
[0, 1, 1, 0, 1, 0, 0]
[1, 0, 0, 0]
[1, 1, 1, 0, 1, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 1, 0, 0]
[1, 0, 0, 1, 1, 0, 0]
[0, 1, 0, 0]
[0, 1, 0, 1, 1, 0, 0]
[1, 1, 0, 0]
[1, 1, 0, 1, 1, 0, 0]
[1, 1, 0, 0]
[0, 0, 1,

volume 96 10 0.133333333333333 8 7
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 80 10 0.150000000000000 9 8
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 48 10 0.166666666666667 10 9
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 32 10 0.183333333333333 11 10
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 32 10 0.200000000000000 12 11
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 32 15 0.216666666666667 13 12
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 48 15 0.233333333333333 14 13
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 64 15 0.250000000000000 15 14
[3, 2, 2, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 96 15 0.266666666666667 16 15
[3, 2, 2, 1]

volume 48 80 1.36666666666667 82 81
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 32 85 1.38333333333333 83 82
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 32 85 1.40000000000000 84 83
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 32 85 1.41666666666667 85 84
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 48 85 1.43333333333333 86 85
[1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 64 85 1.45000000000000 87 86
[3, 2, 2, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 96 90 1.46666666666667 88 87
[3, 2, 2, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 112 90 1.48333333333333 89 88
[3, 2, 2, 1] [1, 1, 1, 1, 1, 1, 1, 1] []
n =  8
A = 510510
P_n =  30030
volume 112 90 1.50000000000000 90 89
[3, 2, 2, 1] [1, 1, 1, 1, 1, 1, 1, 1] 

n =  8
A = 23205
P_n =  30030
volume 112 5 0.0583333333333333 7 134
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 96 10 0.133333333333333 8 135
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 80 10 0.150000000000000 9 136
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 48 10 0.166666666666667 10 137
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 32 10 0.183333333333333 11 138
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 32 10 0.200000000000000 12 139
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 32 15 0.216666666666667 13 140
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 48 15 0.233333333333333 14 141
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 64 15 0.250000000000000 15 142
[1, 2, 4, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  300

volume 112 50 0.850000000000000 102 229
[1, 4, 1, 2] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 112 50 0.858333333333333 103 230
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 96 105 1.73333333333333 104 231
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 80 105 1.75000000000000 105 232
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 48 105 1.76666666666667 106 233
[2, 1, 1, 2, 1, 1] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 32 105 1.78333333333333 107 234
[1, 4, 1, 2] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 32 55 0.900000000000000 108 235
[1, 4, 1, 2] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 32 55 0.908333333333333 109 236
[1, 4, 1, 2] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 48 55 0.916666666666667 110 237
[1, 4, 1, 2] [2, 1, 1, 2, 1, 1] []
n =  8
A = 23205
P_n =  30030
volume 64 55 0.925000000000000 111 238

n =  8
A = 273
P_n =  30030
volume 48 10 0.166666666666667 10 265
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 32 10 0.183333333333333 11 266
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 32 10 0.200000000000000 12 267
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 32 15 0.216666666666667 13 268
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 48 15 0.233333333333333 14 269
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 64 15 0.250000000000000 15 270
[1, 7] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 96 15 0.266666666666667 16 271
[1, 7] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 112 15 0.283333333333333 17 272
[1, 7] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 112 20 0.300000000000000 18 273
[1, 7] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 112 20 0.316666666666667 19 274
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 96 10 0.166666666666667 20 275
[2, 2

n =  8
A = 273
P_n =  30030
volume 32 120 1.98333333333333 119 374
[1, 7] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 32 120 2.00000000000000 120 375
[1, 7] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 32 120 2.01666666666667 121 376
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 48 120 2.03333333333333 122 377
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 64 125 2.05000000000000 123 378
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 96 125 2.06666666666667 124 379
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 112 125 2.08333333333333 125 380
[1, 2, 2, 1, 1, 1] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 112 65 1.05000000000000 126 381
[1, 2, 2, 1, 1, 1] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 112 65 1.05833333333333 127 382
[2, 2, 2, 2] [2, 2, 2, 2] []
n =  8
A = 273
P_n =  30030
volume 96 130 2.13333333333333 128 383
[1, 7] [2, 2, 2, 2] []
3 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

n =  8
A = 91
P_n =  30030
volume 32 15 0.216666666666667 13 396
[4, 2, 2] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 48 15 0.233333333333333 14 397
[4, 2, 2] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 64 15 0.250000000000000 15 398
[1, 1, 6] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 96 15 0.266666666666667 16 399
[1, 1, 6] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 112 15 0.283333333333333 17 400
[1, 1, 6] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 112 20 0.300000000000000 18 401
[1, 1, 6] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 112 20 0.316666666666667 19 402
[4, 2, 2] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 96 10 0.166666666666667 20 403
[4, 2, 2] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 80 10 0.175000000000000 21 404
[4, 2, 2] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 48 10 0.183333333333333 22 405
[4, 2, 2] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 32 10 0.191666666666667 23 406
[4, 2, 2] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
vo

n =  8
A = 91
P_n =  30030
volume 112 125 2.08333333333333 125 508
[1, 1, 1, 2, 1, 1, 1] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 112 65 1.05000000000000 126 509
[1, 1, 1, 2, 1, 1, 1] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 112 65 1.05833333333333 127 510
[4, 2, 2] [4, 2, 2] []
n =  8
A = 91
P_n =  30030
volume 96 130 2.13333333333333 128 511
[1, 1, 6] [4, 2, 2] []


'/home/orges/notebooks/algorithmic_python_music/Tempi_for_two_pianos.xml'

In [40]:
cumSum([1,2,4,8,4,8,4,8,16,8,16,8,4,8,4,4,4,2,8,4,2,1])
#2**lB+1

[0,
 1,
 3,
 7,
 15,
 19,
 27,
 31,
 39,
 55,
 63,
 79,
 87,
 91,
 99,
 103,
 107,
 111,
 113,
 121,
 125,
 127,
 128]

In [85]:
2**lB+1

129